### 展开所有datasets

In [2]:
import fiftyone as fo

datasets = fo.list_datasets()
for dataset_name in datasets:
    print(f"\"{dataset_name}\",")

"my-dataset",
"sahi_null_v2_ms1_0605-0621_40_ok",
"sahi_null_v2_ms1_0710-0726_36_ok",
"sahi_null_v2_ms1_0726-0809_11_ok",
"sahi_null_v2_ms1_0809-0823_34_ok",
"sahi_null_v2_ms2_0726-0809_13_ok",
"sahi_null_v2_ms2_0809-0823_10_ok",
"sahi_null_v2_sw1_0605-0613_07_ok",
"sahi_v1_ms1_0605-0621_40_ok",
"sahi_v1_ms1_0710-0726_36_ok",
"sahi_v1_ms1_0726-0809_11_ok",
"sahi_v1_ms1_0809-0823_34_ok",
"sahi_v1_ms2_0726-0809_13_ok",
"sahi_v1_ms2_0809-0823_10_ok",
"sahi_v1_sw1_0605-0613_07_ok",
"v1_ms1_0710-0726_36_ok",
"v1_ms1_0809-0823_34_ok",
"v1_ms2_0726-0809_13_ok",
"v1_sw1_0605-0613_07_ok",
"v2_ms1_0605-0621_40_ok",
"v2_ms1_0726-0809_11_ok",
"v2_ms2_0809-0823_10_ok",
"v3_sahi_ms1_0605-0621_40_ok",
"v3_sahi_ms1_0710-0726_36_ok",
"v3_sahi_ms1_0726-0809_11_ok",
"v3_sahi_ms1_0809-0823_34_ok",
"v3_sahi_ms2_0726-0809_13_ok",
"v3_sahi_ms2_0809-0823_10_ok",
"v3_sahi_sw1_0605-0613_07_ok",


### 查看单一dataset

In [ ]:
dataset_name = "sahi_null_v2_ms1_0605-0621_40_ok"
dataset = fo.load_dataset(dataset_name)
dataset

### 根据GT和多个Prediction__evaluate__模型在不同dataset表现__保存__到fiftyone里

In [ ]:
import fiftyone as fo
from fiftyone import ViewField as F

datasets_list = [
    "sahi_null_v2_ms1_0605-0621_40_ok",
    # "sahi_null_v2_ms1_0710-0726_36_ok",
    # "sahi_null_v2_ms1_0726-0809_11_ok",
    # "sahi_null_v2_ms1_0809-0823_34_ok",
    # "sahi_null_v2_ms2_0726-0809_13_ok",
    # "sahi_null_v2_ms2_0809-0823_10_ok",
    # "sahi_null_v2_sw1_0605-0613_07_ok",
]

GT_FIELD = "ground_truth"

# ✅ PRED_FIELD 改为 list：批量跑多个 prediction 字段
PRED_FIELDS = [
    "small_slices_a03_yolo11n_custom7null_cv1_ms2_0809-0823_10_ok_8",
    "small_slices_a04_yolo11n_custom7null_cv1_ms2_0809-0823_10_ok_8",
    "small_slices_a03_yolo11s_custom7null_cv1_ms2_0809-0823_10_ok_4",

    "small_slices_a03_yolo11s_custom7null_cv1_ms2_0809-0823_10_ok_16",
    "small_slices_a03_yolo11s_custom7null_cv1_ms2_0809-0823_10_ok_8",
    "small_slices_a04_yolo11s_custom7null_cv1_ms2_0809-0823_10_ok_16",
    "small_slices_a04_yolo11n_custom7null_cv1_ms2_0809-0823_10_ok_4",
    "small_slices_a04_yolo11s_custom7null_cv1_ms2_0809-0823_10_ok_8",
]

IOU = 0.5
METHOD = "coco"
CLASSWISE = True
COMPUTE_MAP = True

confidence_thresholds = [
    0, 0.3, 0.5, 0.6, 0.7, 0.8, 0.9, 0.91, 0.92
]

def safe_tag(name: str, max_len: int = 40) -> str:
    """
    用于把 pred_field 名字压缩成 eval_key 里更安全的短标签（避免太长/含特殊字符）。
    你也可以按需改成更“可读”的规则。
    """
    s = name.replace("/", "_").replace(" ", "_").replace("-", "_")
    return s

for dataset_name in datasets_list:
    print(f"\n📦 Dataset: {dataset_name}")
    dataset = fo.load_dataset(dataset_name)

    # 字段 schema：用于检查 GT/PRED 字段是否存在
    schema_keys = set(dataset.get_field_schema().keys())

    if GT_FIELD not in schema_keys:
        print(f"  ✗ GT_FIELD '{GT_FIELD}' 不存在于该数据集字段中，跳过")
        continue

    for pred_field in PRED_FIELDS:
        if pred_field not in schema_keys:
            print(f"\n🔍 Pred field: {pred_field}")
            print(f"  ✗ 该 pred_field 不存在于数据集字段中，跳过")
            continue

        print(f"\n🔍 Pred field: {pred_field}")

        # （可选）如果想覆盖旧 eval key，可在这里删同前缀的 eval
        # prefix = f"fresh_{safe_tag(pred_field)}_"
        # for k in list(dataset.list_evaluations()):
        #     if k.startswith(prefix):
        #         dataset.delete_evaluation(k)

        pred_tag = safe_tag(pred_field)

        for conf_thresh in confidence_thresholds:
            eval_key = f"fresh_{pred_tag}_conf_{conf_thresh*100:02.0f}"

            try:
                # 1) 过滤预测框（只过滤 label，不丢样本）
                view = dataset.filter_labels(
                    pred_field,
                    F("confidence") > conf_thresh,
                    only_matches=True,
                )

                # 2) 全新评估
                results = view.evaluate_detections(
                    pred_field,
                    gt_field=GT_FIELD,
                    eval_key=eval_key,
                    method=METHOD,
                    iou=IOU,
                    classwise=CLASSWISE,
                    compute_mAP=COMPUTE_MAP,
                )

                m = results.metrics()
                msg = (
                    f"  conf>{conf_thresh:0.2f} | "
                    f"P={m.get('precision', 0):.3f} "
                    f"R={m.get('recall', 0):.3f} "
                    f"F1={m.get('f1_score', 0):.3f}"
                )

                # mAP（兼容写法）
                if hasattr(results, "mAP"):
                    try:
                        msg += f" mAP={results.mAP():.3f}"
                    except Exception:
                        pass

                print(msg)

            except Exception as e:
                print(f"  ✗ {eval_key} failed: {e}")



📦 Dataset: sahi_null_v2_ms1_0605-0621_40_ok

🔍 Pred field: small_slices_a03_yolo11n_custom7null_cv1_ms2_0809-0823_10_ok_8
Evaluating detections...
 100% |█████████████████| 243/243 [1.9s elapsed, 0s remaining, 87.7 samples/s]          
Performing IoU sweep...
 100% |█████████████████| 243/243 [1.2s elapsed, 0s remaining, 142.1 samples/s]         
  conf>0.00 | P=0.371 R=0.999 F1=0.000 mAP=0.850
Evaluating detections...
 100% |█████████████████| 243/243 [1.9s elapsed, 0s remaining, 95.4 samples/s]          
Performing IoU sweep...
 100% |█████████████████| 243/243 [1.3s elapsed, 0s remaining, 140.5 samples/s]         
  conf>0.30 | P=0.432 R=0.999 F1=0.000 mAP=0.850
Evaluating detections...
 100% |█████████████████| 243/243 [1.8s elapsed, 0s remaining, 94.7 samples/s]          
Performing IoU sweep...
 100% |█████████████████| 243/243 [1.3s elapsed, 0s remaining, 136.3 samples/s]         
  conf>0.50 | P=0.477 R=0.995 F1=0.000 mAP=0.850
Evaluating detections...
 100% |█████████████████

### fiftyone的evaluate结果__导出__xlsx文件

In [ ]:
import fiftyone as fo
import pandas as pd
import numpy as np


def get_evaluation_data_like_gui(dataset, eval_key):
    """模拟GUI中的数据获取方式"""
    results = dataset.load_evaluation_results(eval_key)
    info = dataset.get_evaluation_info(eval_key)

    # 获取基础指标
    metrics = results.metrics()

    # 计算平均置信度（GUI中的逻辑）
    avg_confs = {}
    if hasattr(results, "confs") and hasattr(results, "ypred"):
        counts = {}
        sums = {}
        for yp, conf in zip(results.ypred, results.confs):
            if yp not in counts:
                counts[yp] = 0
                sums[yp] = 0.0
            counts[yp] += 1
            sums[yp] += conf if conf is not None else 0.0

        for c in results.classes:
            avg_confs[c] = sums[c] / counts[c] if counts.get(c, 0) > 0 else 0.0

    # 计算总体平均置信度
    if avg_confs:
        metrics["average_confidence"] = np.mean(list(avg_confs.values()))

    # 获取TP/FP/FN
    tp = (
        dataset.sum(f"{eval_key}_tp")
        if f"{eval_key}_tp" in dataset.get_field_schema()
        else 0
    )
    fp = (
        dataset.sum(f"{eval_key}_fp")
        if f"{eval_key}_fp" in dataset.get_field_schema()
        else 0
    )
    fn = (
        dataset.sum(f"{eval_key}_fn")
        if f"{eval_key}_fn" in dataset.get_field_schema()
        else 0
    )

    metrics["tp"] = tp
    metrics["fp"] = fp
    metrics["fn"] = fn

    # 尝试获取mAP/mAR
    try:
        metrics["map"] = results.mAP()
    except:
        metrics["map"] = None

    try:
        metrics["mar"] = results.mAR()
    except:
        metrics["mar"] = None

    return metrics, info




In [ ]:
# 使用示例
# dataset = fo.load_dataset("sahi_null_v2_ms1_0605-0621_40_ok")
datasets_list = [
    "sahi_null_v2_ms1_0605-0621_40_ok",
    "sahi_null_v2_ms1_0710-0726_36_ok",
    "sahi_null_v2_ms1_0726-0809_11_ok",
    "sahi_null_v2_ms1_0809-0823_34_ok",
    "sahi_null_v2_ms2_0726-0809_13_ok",
    "sahi_null_v2_ms2_0809-0823_10_ok",
    "sahi_null_v2_sw1_0605-0613_07_ok",
]

for dataset_name in datasets_list:
    print(f"Processing dataset: {dataset_name}")
    dataset = fo.load_dataset(dataset_name)

    eval_keys = dataset.list_evaluations()

    all_data = []
    for eval_key in eval_keys:
        try:
            metrics, info = get_evaluation_data_like_gui(dataset, eval_key)

            row = {
                "eval_key": eval_key,
                "eval_type": info.config.type,
                "eval_method": info.config.method,
                "average_confidence": metrics.get("average_confidence", 0),
                "support": metrics.get("support", 0),
                "accuracy": metrics.get("accuracy", 0),
                "iou_threshold": getattr(info.config, "iou", 0.5),
                "precision": metrics.get("precision", 0),
                "recall": metrics.get("recall", 0),
                "f1_score": metrics.get("fscore", 0),
                "map": metrics.get("map"),
                "mar": metrics.get("mar"),
                "true_positives": metrics.get("tp", 0),
                "false_positives": metrics.get("fp", 0),
                "false_negatives": metrics.get("fn", 0),
            }
            all_data.append(row)

        except Exception as e:
            print(f"处理 {eval_key} 时出错: {e}")

    df = pd.DataFrame(all_data)
    df.to_excel(f"evaluation_{dataset_name}.xlsx", index=False)